# Procesamiento de Datos para Dashboard de Cohorts y Funnel

Este notebook documenta el proceso de carga, limpieza y transformación de datos del Excel.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Cargar datos
excel_path = '../backend/data/Datos_prueba_v3.xlsx'
excel_file = pd.ExcelFile(excel_path)
print(f"Hojas disponibles: {excel_file.sheet_names}")

## 1. Carga de Datos

In [ ]:
# Cargar cada hoja
inversion_df = pd.read_excel(excel_path, sheet_name=0)
desarrollos_df = pd.read_excel(excel_path, sheet_name=1)
leads_df = pd.read_excel(excel_path, sheet_name=2)

print(f"Inversión: {len(inversion_df)} registros")
print(f"Desarrollos: {len(desarrollos_df)} registros")
print(f"Leads: {len(leads_df)} registros")

In [ ]:
# Ver estructura de cada hoja
print("=== INVERSIÓN ===")
print(inversion_df.info())
print(inversion_df.head())

In [ ]:
print("=== DESARROLLOS ===")
print(desarrollos_df.info())
print(desarrollos_df.head())

In [ ]:
print("=== LEADS ===")
print(leads_df.info())
print(leads_df.head())

## 2. Limpieza de Datos

In [ ]:
# Normalizar nombres de columnas
def normalize_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    return df

inversion_df = normalize_columns(inversion_df)
desarrollos_df = normalize_columns(desarrollos_df)
leads_df = normalize_columns(leads_df)

print("Columnas normalizadas:")
print(f"Inversión: {list(inversion_df.columns)}")
print(f"Desarrollos: {list(desarrollos_df.columns)}")
print(f"Leads: {list(leads_df.columns)}")

In [ ]:
# Convertir columnas de fecha
date_columns = [col for col in leads_df.columns if 'fecha' in col.lower()]
print(f"Columnas de fecha encontradas: {date_columns}")

for col in date_columns:
    leads_df[col] = pd.to_datetime(leads_df[col], errors='coerce')
    print(f"{col}: {leads_df[col].dtype}")

## 3. Cálculo de Semanas ISO (Cohorts)

In [ ]:
# Encontrar columna de fecha de registro
registro_col = [col for col in leads_df.columns if 'registro' in col.lower()]
if registro_col:
    registro_col = registro_col[0]
    print(f"Usando columna: {registro_col}")
else:
    print("No se encontró columna de registro")

In [ ]:
# Calcular semana ISO
if registro_col:
    valid_dates = leads_df[registro_col].notna()
    iso_calendar = leads_df.loc[valid_dates, registro_col].dt.isocalendar()
    
    leads_df['year_iso'] = None
    leads_df['week_iso'] = None
    leads_df.loc[valid_dates, 'year_iso'] = iso_calendar.year.values
    leads_df.loc[valid_dates, 'week_iso'] = iso_calendar.week.values
    
    leads_df['cohort_week'] = leads_df.apply(
        lambda x: f"{int(x['year_iso'])}-W{int(x['week_iso']):02d}" 
        if pd.notna(x['year_iso']) else None,
        axis=1
    )
    
    print(f"Cohorts únicos: {leads_df['cohort_week'].nunique()}")
    print(leads_df['cohort_week'].value_counts().head(10))

## 4. Análisis de Funnel

In [ ]:
# Identificar columnas del funnel
funnel_stages = {
    'Lead': registro_col if registro_col else None,
    'Contacto': next((c for c in leads_df.columns if 'contacto' in c.lower()), None),
    'Cita': next((c for c in leads_df.columns if 'cita' in c.lower()), None),
    'Venta': next((c for c in leads_df.columns if 'venta' in c.lower()), None),
    'Escrituración': next((c for c in leads_df.columns if 'escritur' in c.lower()), None)
}

print("Columnas del funnel:")
for stage, col in funnel_stages.items():
    print(f"  {stage}: {col}")

In [ ]:
# Calcular métricas del funnel
total_leads = len(leads_df)
print(f"\nMétricas del Funnel:")
print(f"Total Leads: {total_leads:,}")

for stage, col in funnel_stages.items():
    if col and col in leads_df.columns:
        if stage == 'Lead':
            count = total_leads
        else:
            count = leads_df[col].notna().sum()
        pct = count / total_leads * 100
        print(f"{stage}: {count:,} ({pct:.1f}%)")

## 5. Análisis de Cohorts

In [ ]:
# Calcular conversiones por cohort
if 'cohort_week' in leads_df.columns:
    cohort_summary = leads_df.groupby('cohort_week').agg({
        registro_col: 'count' if registro_col else 'size'
    }).rename(columns={registro_col: 'leads'})
    
    for stage, col in funnel_stages.items():
        if col and col in leads_df.columns and stage != 'Lead':
            cohort_summary[stage.lower()] = leads_df.groupby('cohort_week')[col].apply(
                lambda x: x.notna().sum()
            )
    
    print("Resumen por cohort (primeros 10):")
    print(cohort_summary.head(10))

## 6. Resumen

In [ ]:
print("="*50)
print("RESUMEN DEL PROCESAMIENTO")
print("="*50)
print(f"Total de leads: {len(leads_df):,}")
print(f"Desarrollos: {desarrollos_df.shape[0]}")
print(f"Registros de inversión: {len(inversion_df):,}")
print(f"Cohorts (semanas): {leads_df['cohort_week'].nunique()}")
print(f"Rango de fechas: {leads_df[registro_col].min()} a {leads_df[registro_col].max()}")